Statistik Dasar

In [2]:
import pandas as pd

data = pd.read_csv("data/data_train.csv")

In [ ]:
#Statistik Dasar


In [ ]:
#Duplicate Value

In [ ]:
#Missing Value

In [ ]:
#Outlier

In [ ]:
#Distribusi Data dan Histogram

In [ ]:
#Korelasi dengan Kolom Target